In [1]:
from dotenv import load_dotenv,find_dotenv
from typing import Optional,List

from langchain_core.pydantic_v1 import BaseModel, Field
_ = load_dotenv(find_dotenv())
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-4o-mini")


In [8]:
class Classifier(BaseModel):
    sentiment: Optional[str] = Field(None, description="Sentiment of the text")
    language: Optional[str] = Field(None, description="Language of the text")
    political_tendency: Optional[str] = Field(None, description="Political Tendency of the text")

In [9]:
from langchain_core.prompts import ChatPromptTemplate

# Fix the Classifier class definition
class Classifier(BaseModel):
    sentiment: Optional[str] = Field(None, description="Sentiment of the text")
    language: Optional[str] = Field(None, description="Language of the text")
    political_tendency: Optional[str] = Field(None, description="Political Tendency of the text")

prompt=ChatPromptTemplate.from_template("""
Extract the sentiment, language and political tendency of the text.
Text {text}                                 
""")
chain = prompt | chatbot.with_structured_output(schema=Classifier)
trump_follower = "I'm confident that President Trump's leadership and track record will once again resonate with Americans. His strong stance on economic growth and national security is exactly what our country needs at this pivotal moment. We need to bring back the proven leadership that can make America great again!"
chain.invoke({"text":trump_follower})

Classifier(sentiment='positive', language='English', political_tendency='conservative')

In [10]:
biden_follower = "I believe President Biden's compassionate and steady approach is vital for our nation right now. His commitment to healthcare reform, climate change, and restoring our international alliances is crucial. It's time to continue the progress and ensure a future that benefits all Americans."
chain.invoke({"text":biden_follower})


Classifier(sentiment='positive', language='English', political_tendency='liberal')

In [11]:
es="Creo que el enfoque compasivo y firme del presidente Biden es vital para nuestra nación en este momento. Su compromiso con la reforma sanitaria, el cambio climático y el restablecimiento de nuestras alianzas internacionales es crucial. Es hora de continuar el progreso y garantizar un futuro que beneficie a todos los estadounidenses."
chain.invoke({"text":es})

Classifier(sentiment='positivo', language='español', political_tendency='liberal')

In [13]:
class Classifier_adv(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"],description="Sentiment of the text") # ... means it is mandatory
    language: str = Field(..., description="Language of the text", enum=["en", "es"])
    political_tendency: str = Field(..., description="Political Tendency of the text", enum=["left", "right"])
chain_adv = prompt | chatbot.with_structured_output(schema=Classifier_adv)
chain_adv.invoke({"text":biden_follower})

Classifier_adv(sentiment='happy', language='en', political_tendency='left')

In [14]:
italian="Credo che l’approccio compassionevole e costante del presidente Biden sia vitale per la nostra nazione in questo momento. Il suo impegno per la riforma sanitaria, il cambiamento climatico e il ripristino delle nostre alleanze internazionali è cruciale. È tempo di continuare il progresso e garantire un futuro a beneficio di tutti gli americani."
chain_adv.invoke({"text":italian}) # This will fail as the language is not supported or language is not correct

Classifier_adv(sentiment='happy', language='es', political_tendency='left')